# Uso PySpark con Tablas Delta y Archivos Parquet

In [1]:
# Mapeo el path abfss de la tabla que voy a consultar 

workspace= '0bdb98c9-b5dd-4510-a075-30c8479eff94'
dfs= '@onelake.dfs.fabric.microsoft.com'
lakehouse= '1a8a18b8-3140-4c95-9635-8e12298042c3'
schema= 'f1'
table_name= 'drivers'

path= 'abfss://' + workspace + dfs + '/' + lakehouse + '/Tables/' + schema + '/' + table_name

print(path)

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 3, Finished, Available, Finished)

abfss://0bdb98c9-b5dd-4510-a075-30c8479eff94@onelake.dfs.fabric.microsoft.com/1a8a18b8-3140-4c95-9635-8e12298042c3/Tables/f1/drivers


In [2]:
# Armo dataframe leyendo tabla delta
df_drivers= spark.read.load(path)

display(df_drivers)

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e1288b5b-0054-48bb-9ed7-75151f784a59)

In [28]:
# Guardo el dataframe en Onelake 
lh_path= 'abfss://Richard@onelake.dfs.fabric.microsoft.com/LH_Richard.Lakehouse/Files/F1/drivers'

df_drivers.write.mode("overwrite").format("parquet").save(lh_path)

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 30, Finished, Available, Finished)

Puedo armar dataframes haciendo queries a las tablas delta

In [3]:
# Armo dataframe leyendo tabla delta
df_calendario= spark.sql('SELECT * FROM LH_Richard.masters.calendario order by Fecha')

# Muestro el dataframe
display(df_calendario)

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 968ee386-7bd2-4cc0-b5d0-6b0e008ad13b)

In [5]:
# filtro los años 2020 y 2022
df_2020_2022= df_calendario.filter(df_calendario["Year"].isin(2020, 2022))

display(df_2020_2022)

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e511e82d-a072-4860-be80-feefb8db1fb6)

In [6]:
# Guardo el dataframe como una tabla delta 
df_2020_2022.write.mode("overwrite").format("delta").save('Tables/dbo/calendario_2020_2022')

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 8, Finished, Available, Finished)

In [7]:
# Armo dataframe leyendo tabla delta de otro lakehouse
path= 'abfss://Roma@onelake.dfs.fabric.microsoft.com/LH_Roma.Lakehouse/Tables/f1/sessions'

df_sessions= spark.read.format('delta').load(path)

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 9, Finished, Available, Finished)

In [8]:
# Guardo el dataframe en Onelake
df_sessions.write.mode("overwrite").format("parquet").save('Files/F1/sessions')

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 10, Finished, Available, Finished)

In [9]:
display(df_sessions)

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ab1b6853-07e7-423d-b4d5-b65a44470949)

In [10]:
# Mostrar los tipos de datos de cada columna

print("Tipos de datos con dtypes:")
df_sessions.printSchema()

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 12, Finished, Available, Finished)

Tipos de datos con dtypes:
root
 |-- meeting_key: long (nullable = true)
 |-- session_key: long (nullable = true)
 |-- location: string (nullable = true)
 |-- date_start: string (nullable = true)
 |-- date_end: string (nullable = true)
 |-- session_type: string (nullable = true)
 |-- session_name: string (nullable = true)
 |-- country_key: long (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- circuit_key: long (nullable = true)
 |-- circuit_short_name: string (nullable = true)
 |-- gmt_offset: string (nullable = true)
 |-- year: long (nullable = true)



In [11]:
from pyspark.sql.functions import col, to_date

# Modifico los tipo de columnas del dataframe
df_sessions = df_sessions.withColumn("date_start", to_date(col("date_start")))
df_sessions = df_sessions.withColumn("date_end", to_date(col("date_end")))
df_sessions = df_sessions.withColumn("gmt_offset", to_date(col("gmt_offset")))

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 13, Finished, Available, Finished)

In [38]:
# Guardo el dataframe como tabla delta
df_sessions.write.mode('overwrite').format('delta').save('Tables/F1/sessions')

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 40, Finished, Available, Finished)

Armo dataframe leyendo un json

In [12]:
path_json = 'abfss://GOMC@onelake.dfs.fabric.microsoft.com/LH_GOMC.Lakehouse/Files/TC/Periodos/tc-2022-01-01 al 2022-12-31.json'

df_tc_2022 = spark.read.option("multiline", "true").json(path_json)

display(df_tc_2022)

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a7cc362e-eea2-452c-a8ca-e5be0b5cdd9e)

In [96]:
from pyspark.sql.functions import col, lit, struct
from pyspark.sql import functions as F

# Lista de monedas que quieres extraer
monedas = ['ARS', 'BRL', 'EUR', 'CLP', 'PYG', 'UYU', 'COP', 'MXN', 'PEN']

# Obtener las claves del struct (fechas) desde el esquema
fechas = df_tc_2022.select("rates").schema[0].dataType.fieldNames()

# Inicializamos una lista para almacenar los DataFrames parciales
dataframes = []

# Iteramos por cada fecha en las claves del struct
for fecha in fechas:
    # Seleccionamos la base y las tasas de la fecha actual
    df_tmp = df_tc_2022.select(
        col("base"),
        lit(fecha).alias("Fecha"),  # Usamos la fecha como una nueva columna
        *[col(f"rates.`{fecha}`.{moneda}").alias(moneda) for moneda in monedas]
    )
    
    # Añadimos el DataFrame temporal a la lista
    dataframes.append(df_tmp)

# Unir los DataFrames parciales uno por uno
df_unido = dataframes[0]
for df in dataframes[1:]:
    df_unido = df_unido.unionByName(df)

# Agregar la columna del año
df_final = df_unido.withColumn("year", F.year("Fecha"))

# Mostrar el resultado
display(df_final)

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 98, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9ec843a1-3de0-4b32-9c5c-6b0580a918cf)

In [97]:
df_final.write.mode('overwrite').format('delta').save('Tables/dbo/TC2')

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 99, Finished, Available, Finished)

In [107]:
%%sql
create or replace table LH_Richard.dbo.tc_2021
as
select *
from json.`abfss://Richard@onelake.dfs.fabric.microsoft.com/LH_Richard.Lakehouse/Files/calendario/tc-2024.json`


StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 109, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

Puedo armar dataframes leyendo archivos parquet en Onelake

In [46]:
# Mapeo las tablas Delta 
onelake_path= 'abfss://Richard@onelake.dfs.fabric.microsoft.com/LH_Richard.Lakehouse/Files/F1/drivers'

# Armo dataframe
df_drives= spark.read.format('parquet').load(onelake_path)

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 48, Finished, Available, Finished)

In [47]:
# Armo tabla temporal con dataframe

df_drives.createOrReplaceTempView('temp_drivers')

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 49, Finished, Available, Finished)

# Uso Spark SQL con tablas Delta y tablas temporales

In [49]:
%%sql
SELECT *
from temp_drivers
Limit 5

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 51, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 6 fields>

Puedo llamar a tablas delta especificando el Lakehouse y el esquema

In [51]:
%%sql
SELECT *
FROM LH_Richard.masters.TC
where Fecha >= date_add(current_date,-30)
order by Fecha DESC

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 53, Finished, Available, Finished)

<Spark SQL result set with 25 rows and 4 fields>

In [88]:
%%sql
-- Creo tabla delta con una query sobre la tabla delta

CREATE TABLE IF NOT EXISTS LH_Richard.dbo.tc_2020_2022
As
SELECT *
FROM LH_Richard.masters.tc
where Fecha between '2020-01-01' and '2022-12-31'

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 90, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [74]:
%%sql
-- Creo tabla delta con una query sobre la tabla delta

CREATE TABLE IF NOT EXISTS LH_Richard.dbo.calendario_2020_2022
As
SELECT *
FROM LH_Richard.masters.calendario
where Fecha between '2020-01-01' and '2022-12-31'

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 76, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [76]:
%%sql
SELECT Year,
       count(*)
FROM LH_Richard.dbo.calendario_2020_2022
GROUP BY Year

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 78, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 2 fields>

In [77]:
%%sql
-- Borro datos de una tabla Delta
delete from LH_Richard.dbo.calendario_2020_2022
where Fecha <= '2020-12-31'

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 79, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [20]:
%%sql
Update LH_Richard.dbo.calendario_2020_2022
set 
    Year= 9999
where Fecha = '2022-07-08'

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 22, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [82]:
%%sql
-- Puedo ver las diferentes versiones de la tabla Delta

DESCRIBE history LH_Richard.dbo.calendario_2020_2022

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 84, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 15 fields>

In [21]:
# Si especifico la versión puedo usar su Delta_log para leer datos de versiones anteriores
df__old= spark.read.format("delta").option('VERSION AS OF', 3).load("Tables/dbo/calendario_2020_2022")
df__old= df__old.filter(df__old.Fecha== '2022-07-08' )
display(df__old)

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cebd30cc-a6de-4890-a38c-3dce9608d698)

In [22]:
%%sql
-- Si especifico la versión puedo usar su Delta_log para leer datos de versiones anteriores

select *
from LH_Richard.dbo.calendario_2020_2022 VERSION AS OF 1
where Fecha = '2022-07-08'

StatementMeta(, 80197acd-2364-4602-a342-c145266190c9, 24, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 7 fields>

# Uso MERGE para hacer UPDATE, INSERT y DELETE en tablas DELTA a partir de una condicion

In [98]:
%%sql
-- Actualizo la tabla target usando otra tabla delta usando la Fecha como join

MERGE INTO LH_Richard.dbo.TC2    					         AS target  
    USING 
		(SELECT *
        FROM LH_Richard.masters.TC
        WHERE Fecha BETWEEN '2020-01-01' and '2022-01-03')    AS source -- puedo usar una query o una tabla delta

        ON target.Fecha = source.Fecha 	--condicion que se evalua entre las dos tablas delta

    -- Actualiza cuando se cumple la condición
    WHEN MATCHED THEN
        UPDATE SET 
             target.base = 'modificado'
            ,target.EUR = source.EUR
  
    -- Inserto registros cuando no se cumple la condición
    WHEN NOT MATCHED THEN
        INSERT 
            (Fecha,
            base,
			EUR,
			year)

        VALUES
            (source.Fecha,
            source.base,
			source.EUR,
			source.year)

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 100, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [101]:
%%sql
-- hago un UPSERT sobre los tablas de la tabla target

MERGE INTO LH_Richard.dbo.calendario_2020_2022                AS target
  USING (SELECT *
        FROM LH_Richard.masters.calendario
        WHERE Fecha BETWEEN '2021-01-01' and '2021-01-03')    AS source --Puedo utilizar una query como tabla origen
  ON target.Fecha = source.Fecha            --condicion que se evalua entre las dos tablas delta

-- Actualizao los registros cuando se cumple la condición
  WHEN MATCHED THEN
    UPDATE SET
      target.Periodo = 'Modificado'

-- Elimino registros cuando no se cumple la condición
  WHEN NOT MATCHED BY SOURCE THEN
    DELETE

StatementMeta(, 91e070b2-d05f-45b3-82a7-cbb2cbe13ec6, 103, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>